# NTRU IND_CCA

In [43]:
#DEFINIÇÕES GERAIS

#d>6 -> parâmetro de segurança
d=4
#N -> um primo maior ou igual 2^d
N = next_prime(1 << d) 
#p -> um pequeno primo 3, 5 ou 7
p=3
#q -> um primo da ordem de grandeza de $p*N$
q = next_prime(p*N)

Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
R.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

#mensagem
m = vector_generator()

print(d, q, p, N)
print ('\nmensagem = ',m)


4 53 3 17

mensagem =  [1, 1, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 1, -1, 1]


In [63]:
import hashlib, hmac
from sage.crypto.util import ascii_integer
from random import choice, randint


def vector_generator(): 
    return [choice([-1,0,1]) for k in range(N)]


# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ;
    ll = map(lift,f.list()) 
    return[n if n <= qq else n-q for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]



class NTRU(object):
    def __init__(self):
        # calcular um 'f' invertível
        f = R(0)
        while not f.is_unit():
            F = R(vector_generator());
            f = 1 + p*F
        # gerar as chaves
        G = R(vector_generator()) ;
        g = p*G
        self.g = g
        self.f = f
        self.h = f^(-1) * g
        
        
    def encrypt(self,m):
        r = R(vector_generator())
        return r*self.h + R(m)
    
    
    def decrypt(self,c):
        a = c*self.f
        return prnd(qrnd(a))

In [64]:
# Construção da matriz geradora por blocos
import numpy as np
# http://doc.sagemath.org/html/en/reference/modules/sage/modules/free_module_integer.html
import sage.crypto.lattice as lat
import sage.modules.free_module_integer as fmi


# rodar um vetor
def rotate(x):
    matrizX = [x]
    for k in range(N-1):
        x = [x[-1]] + x[:-1]
        matrizX = matrizX + [x]
    # shift right rotate
    matrizX = matrix(ZZ,N,N,matrizX)
    return matrizX


class Lat(NTRU):
    
    def __init__(self):
        N2 = NTRU()
        
        self.m = m
        self.c = N2.encrypt(m)
        
        print('criptograma = ',c)

        matrizI = identity_matrix(ZZ,N);
        matrizQ = q*matrizI
        matriz0 = matrix(ZZ,N,N,[0]*(N^2))

        #chave pública h
        h=N2.h
        h = qrnd(h)

        matrizH = rotate(h)

        self.G = block_matrix([[matrizI,matrizH],[matriz0,matrizQ]])

    #-----------------------------------------------------------------#
    
    def retrieve(self):
        
        #Cálculo do CVP
        LL = matrix(self.G.LLL())
        
        target_R = self.c.list()
        target_R = [lift(a) for a in target_R]
        for i in range(N):
            target_R.insert(0,0)
        target=[0]*(2*N)
        
        target = matrix(1,2*N,target_R)
        zero = matrix(2*N,1,[0]*(2*N))
        M = matrix(1,1,[(N*q)**10])
        
        L1 = block_matrix(2,2,[[LL,zero],[target,M]])
        L1_red = L1.LLL()
        
        self.erro = np.array(L1_red[2*N][:-1])
        Mensagem=self.erro[N:]
        y = self.erro + target_R

        print("Solução aproximada")
        print("erro   = ", tuple(self.erro))
        print("vector = ",tuple(y))
        print('Mensagem original= ', self.m)
        print('Mensagem recuperada resolvendo o CVP= ', Mensagem)
        
        resolvido =True
        for i in self.m:
            if self.m[i]!= Mensagem[i]:
                resolvido = False
                return -1
        
        print("\n\nMensagem recuperada = mensagem original?",resolvido)
        
        short1 = np.array(L1_red[0][:-1])
        print("\n\nshort aproxi = ",tuple(short1))
        f = short1[:N] ;
        g = short1[N:]
        print('f recuperado = ',f)
        print('g recuperado = ',g)
        

In [65]:
L=Lat()

criptograma =  362*x^130 + 253*x^129 + 186*x^128 + 273*x^127 + 197*x^126 + 207*x^125 + 148*x^124 + 381*x^123 + 363*x^122 + 117*x^121 + 156*x^120 + 302*x^119 + 148*x^118 + 358*x^117 + 247*x^116 + 126*x^115 + 283*x^114 + 102*x^113 + 94*x^112 + 361*x^111 + 201*x^110 + 121*x^109 + 241*x^108 + 119*x^107 + 276*x^106 + 340*x^105 + 345*x^104 + 385*x^103 + 154*x^102 + 212*x^101 + 228*x^100 + 363*x^99 + 6*x^98 + 141*x^97 + 109*x^96 + 388*x^95 + 211*x^94 + 43*x^93 + 259*x^92 + 162*x^91 + 138*x^90 + 390*x^89 + 382*x^88 + 357*x^87 + 166*x^86 + 8*x^85 + 37*x^84 + 235*x^83 + 266*x^82 + 300*x^81 + 338*x^80 + 293*x^79 + 20*x^78 + 164*x^77 + 261*x^76 + 157*x^75 + 128*x^74 + 159*x^73 + 246*x^72 + 61*x^71 + 165*x^70 + 373*x^69 + 149*x^68 + 244*x^67 + 50*x^66 + 238*x^65 + 132*x^64 + 116*x^63 + 223*x^62 + 206*x^61 + 189*x^60 + 314*x^59 + 201*x^58 + 89*x^57 + 274*x^56 + 228*x^55 + 330*x^54 + 83*x^53 + 257*x^52 + 294*x^51 + 232*x^50 + 303*x^49 + 19*x^48 + 12*x^47 + 54*x^46 + 209*x^45 + 230*x^44 + 293*x^43 + 1

In [66]:
L.retrieve()

Solução aproximada
erro   =  (-1, -1, 1, 1, 0, 0, -1, 0, 1, 0, 0, 1, 1, -1, 0, -1, 1, 1, 1, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 1, -1, 1)
vector =  (-1, -1, 1, 1, 0, 0, -1, 0, 1, 0, 0, 1, 1, -1, 0, -1, 1, 26, 40, 3, 3, 2, 31, 42, 35, 19, 53, 9, 38, 15, 40, 38, 1, 21)
Mensagem original=  [1, 1, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 1, -1, 1]
Mensagem recuperada resolvendo o CVP=  [ 1  1 -1  0  0 -1 -1 -1  0  1  1  0  0  1  1 -1  1]


Mensagem recuperada = mensagem original? True


short aproxi =  (2, 1, -4, 2, -2, -1, -3, -1, -3, -2, 1, -1, 3, 2, 2, 2, -1, 3, -1, 0, 2, -1, 0, 0, 0, 0, 0, 0, 4, 4, 3, 0, 0, 3)
f recuperado =  [ 2  1 -4  2 -2 -1 -3 -1 -3 -2  1 -1  3  2  2  2 -1]
g recuperado =  [ 3 -1  0  2 -1  0  0  0  0  0  0  4  4  3  0  0  3]
